In [1]:
""" 3/27/18 
With a functional mister_ed package, it's time to run the madry challenge.

Step 1 is to build a defended network that can transfer to their tensorflow nets.
Because Madry uses tensorflow, transferability is a must. 

From Madry:
The model is a residual convolutional neural network consisting of five residual units and a fully connected layer. 
This architecture is derived from the "w32-10 wide" variant of the Tensorflow model repository. 
The network was trained against an iterative adversary that is allowed to perturb each pixel by at most epsilon=8.0.
"""
None

In [2]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval


In [3]:
# Step -1: load the classifier, data_loader, and the normalizer 
use_gpu = True
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()

val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)


Files already downloaded and verified


In [4]:
# Step 0: initialize hyperparams for attack and training 
PGD_L_INF = 8.0 / 255.0
PGD_STEP_SIZE = 1.0 / 255.0
PGD_NUM_ITER = 16
PGD_TRAINING_PROP = 1.0

In [5]:
# Step 1: setup loss object 
standard_xentropy = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer)

In [6]:
# Step 2: build attack object and its parameters
pgd_attack_object = aa.LInfPGD(classifier_net, cifar_normer, standard_xentropy)
pgd_attack_params = advtrain.AdversarialAttackParameters(
                            pgd_attack_object, 
                            PGD_TRAINING_PROP,
                            {'attack_kwargs': 
                                {'l_inf_bound': PGD_L_INF,
                                 'step_size': PGD_STEP_SIZE,
                                 'num_iterations': PGD_NUM_ITER,
                                 'random_init': True,
                                 'signed': True,
                                 'verbose': False}})


In [7]:
# Step 3: build training object, training loss, data loader

pgd_cifar_training = advtrain.AdversarialTraining(classifier_net, 
                                                  cifar_normer, 
                                                  'full_pgd_cifar_madry_params',
                                                  'cifar_resnet32')
train_loss = nn.CrossEntropyLoss()
train_loader = cifar_loader.load_cifar_data('train', normalize=False)



Files already downloaded and verified


In [8]:
# Step 4: Do the training 
'''pgd_cifar_training.train(train_loader, 256, train_loss, 
                         attack_parameters=pgd_attack_params,
                         use_gpu=True, verbosity='high')
                         
'''

"pgd_cifar_training.train(train_loader, 256, train_loss, \n                         attack_parameters=pgd_attack_params,\n                         use_gpu=True, verbosity='high')\n                         \n"

In [9]:
"""3/28/18 
Load and verify the adversarially trained model 

"""

import checkpoints
base_model = cifar_resnets.resnet32()
experiment_name = 'full_pgd_cifar_madry_params'
architecture_name = 'cifar_resnet32'
epoch = 100
print checkpoints.params_to_filename(experiment_name, architecture_name, epoch)
print checkpoints.CHECKPOINT_DIR
adv_trained_net = checkpoints.load_state_dict(experiment_name, architecture_name, epoch, base_model)

full_pgd_cifar_madry_params.cifar_resnet32.000100.path.tar
/home/mjordan/git_repos/mister_ed/pretrained_models/


In [10]:
reload(adveval)
# Now run the evaluation script on the eval set 

#0) initialize the classifier/normalizer/data loader 
cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS, std=config.CIFAR10_STDS)
val_loader = cifar_loader.load_cifar_data('val', normalize=False)

#1) Build some attack objects 
L_INF_BOUND = 8.0 / 255.0
# --- FGSM attack
fgsm_xentropy_loss = plf.VanillaXentropy(adv_trained_net,
                                         normalizer=cifar_normer)
fgsm_attack_obj = aa.FGSM(adv_trained_net, cifar_normer,
                          fgsm_xentropy_loss)
fgsm_spec_params = {'attack_kwargs': {'l_inf_bound': L_INF_BOUND, 'verbose': False}}
fgsm_attack_params = advtrain.AdversarialAttackParameters(
                            fgsm_attack_obj, 0.5, fgsm_spec_params)

# --- PGD attack 
pgd_xentropy_loss = plf.VanillaXentropy(adv_trained_net, normalizer=cifar_normer)
pgd_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, pgd_xentropy_loss)
pgd_spec_params = {'attack_kwargs': {'l_inf_bound': L_INF_BOUND,
                                     'step_size': 1.0/255.0,
                                     'num_iterations': 16,
                                     'random_init': True,
                                     'signed': True,
                                     'verbose': False}}
pgd_attack_params = advtrain.AdversarialAttackParameters(
                        pgd_attack_obj, 0.5, pgd_spec_params)

# 2) Run the evaluation and print outpus 
eval_obj = adveval.AdversarialEvaluation(adv_trained_net, cifar_normer)
#eval_obj.evaluate_ensemble(val_loader, {'fgsm': fgsm_attack_params, 'pgd': pgd_attack_params}, 
#                  use_gpu=True)

Files already downloaded and verified


In [11]:
# Build madry challenge npy 
# 1) Build data loader and new attack params 
reload(adveval)
deterministic_loader = cifar_loader.load_cifar_data('val', normalize=False, shuffle=False, no_transform=True)

pgd_xentropy_loss = plf.VanillaXentropy(adv_trained_net, normalizer=cifar_normer)
pgd_attack_obj = aa.LInfPGD(adv_trained_net, cifar_normer, pgd_xentropy_loss)
pgd_spec_params = {'attack_kwargs': {'l_inf_bound': L_INF_BOUND,
                                     'step_size': 1.0/255.0,
                                     'num_iterations': 16,
                                     'random_init': True,
                                     'signed': True,
                                     'verbose': False}}
pgd_attack_params = advtrain.AdversarialAttackParameters(
                        pgd_attack_obj, 1.0, pgd_spec_params)

# 2) Run attack

eval_obj.full_attack(deterministic_loader, pgd_attack_params, 'pgd_trained', use_gpu=True, verbose=True)

Files already downloaded and verified
Minibatch 00/78
Minibatch 01/78
Minibatch 02/78
Minibatch 03/78
Minibatch 04/78
Minibatch 05/78
Minibatch 06/78
Minibatch 07/78
Minibatch 08/78
Minibatch 09/78
Minibatch 10/78
Minibatch 11/78
Minibatch 12/78
Minibatch 13/78
Minibatch 14/78
Minibatch 15/78
Minibatch 16/78
Minibatch 17/78
Minibatch 18/78
Minibatch 19/78
Minibatch 20/78
Minibatch 21/78
Minibatch 22/78
Minibatch 23/78
Minibatch 24/78
Minibatch 25/78
Minibatch 26/78
Minibatch 27/78
Minibatch 28/78
Minibatch 29/78
Minibatch 30/78
Minibatch 31/78
Minibatch 32/78
Minibatch 33/78
Minibatch 34/78
Minibatch 35/78
Minibatch 36/78
Minibatch 37/78
Minibatch 38/78
Minibatch 39/78
Minibatch 40/78
Minibatch 41/78
Minibatch 42/78
Minibatch 43/78
Minibatch 44/78
Minibatch 45/78
Minibatch 46/78
Minibatch 47/78
Minibatch 48/78
Minibatch 49/78
Minibatch 50/78
Minibatch 51/78
Minibatch 52/78
Minibatch 53/78
Minibatch 54/78
Minibatch 55/78
Minibatch 56/78
Minibatch 57/78
Minibatch 58/78
Minibatch 59/78
Mi

array([[[[150.        , 151.        , 157.        , ..., 129.        ,
          118.        , 119.388565  ],
         [144.        , 143.        , 151.        , ..., 128.        ,
          117.        , 111.        ],
         [143.        , 143.        , 150.        , ..., 132.57864   ,
          122.00001   , 112.        ],
         ...,
         [ 76.        ,  34.        ,  22.999998  , ...,  30.27347   ,
            4.9999995 ,  41.        ],
         [ 60.41706   ,  41.        ,  27.000002  , ...,  22.320187  ,
           20.999998  ,  28.000002  ],
         [ 46.        ,  48.        ,  38.574524  , ...,  15.999999  ,
           26.000002  ,  24.444199  ]],

        [[120.        , 119.        , 124.        , ..., 102.929     ,
           99.        ,  93.        ],
         [120.        , 118.        , 122.        , ..., 103.        ,
           99.        ,  96.        ],
         [118.        , 117.        , 119.        , ..., 106.        ,
          100.730286  ,  84.     

In [ ]:
len(deterministic_loader.dataset)
deterministic_loader.batch_size

In [ ]:
import numpy as np
foo = np.load(os.path.join(config.OUTPUT_IMAGE_PATH, 'pgd_trained.npy'))
foo.shape